In [ ]:
import obi_one as obi
import os
import pandas as pd

In [ ]:
input_root = "/Users/pokorny/Data/Circuits"
output_root = "../../../obi-output/extracted_small_microcircuits_hippocampus"

In [ ]:
circuit = obi.Circuit(name="rCA1-CYLINDER-REF", path=os.path.join(input_root, "rCA1-CYLINDER-REF", "circuit_config_postfix2.json"))

In [ ]:
# Neuron IDs for rCA1-CYLINDER-REF
node_id_lists = [obi.NamedTuple(name="rCA1-CYLINDER-REF-1PC-8PV-01", elements=(5625, 5821, 5847, 5815, 5591, 5619, 5675, 5845, 3673)),
                 obi.NamedTuple(name="rCA1-CYLINDER-REF-1PC-8PV-02", elements=(848, 5827, 5834, 5644, 5822, 5831, 5815, 5690, 5816)),
                 obi.NamedTuple(name="rCA1-CYLINDER-REF-1PC-8PV-03", elements=(2496, 5812, 5836, 5815, 5818, 5834, 5822, 5587, 5827)),
                 obi.NamedTuple(name="rCA1-CYLINDER-REF-1PC-8PV-04", elements=(220, 5832, 5815, 5834, 5819, 5825, 5826, 5841, 5830)),
                 obi.NamedTuple(name="rCA1-CYLINDER-REF-1PC-8PV-05", elements=(295, 5834, 5830, 5822, 5828, 5837, 5831, 5840, 5827)),
                 obi.NamedTuple(name="rCA1-CYLINDER-REF-1PC-8PV-06", elements=(5055, 5826, 5832, 5847, 5830, 5840, 5828, 5822, 5841)),
                 obi.NamedTuple(name="rCA1-CYLINDER-REF-1PC-8PV-07", elements=(2805, 5832, 5826, 5807, 5830, 5825, 5834, 5819, 5842)),
                 obi.NamedTuple(name="rCA1-CYLINDER-REF-1PC-8PV-08", elements=(357, 5819, 5842, 5825, 5826, 5834, 5830, 5840, 5832)),
                 obi.NamedTuple(name="rCA1-CYLINDER-REF-1PC-8PV-09", elements=(220, 5821, 5830, 5834, 5819, 5825, 5832, 5826, 5840)),
                 obi.NamedTuple(name="rCA1-CYLINDER-REF-1PC-8PV-10", elements=(2459, 5834, 5819, 5825, 5842, 5826, 5832, 5840, 5821)),
                ]

In [ ]:
circuit_extractions_scan_config = obi.CircuitExtractionScanConfig(
                    initialize=obi.CircuitExtractionScanConfig.Initialize(
                        circuit=circuit,
                        run_validation=False,
                        do_virtual=True,
                        create_external=True,
                        virtual_sources_to_ignore=("external_hippocampus_neurons__hippocampus_neurons__chemical_synapse", ),
                    ),
                    neuron_set=obi.IDNeuronSet(neuron_ids=node_id_lists)
)

grid_scan = obi.GridScanGenerationTask(form=circuit_extractions_scan_config, output_root=output_root, coordinate_directory_option="VALUE")

In [ ]:
# Display the intermediary data
grid_scan.multiple_value_parameters(display=True)
grid_scan.coordinate_parameters(display=True)
# grid_scan.coordinate_instances(display=True)

In [ ]:
grid_scan.execute()
obi.run_tasks_for_generated_scan(grid_scan)

In [ ]:
# Check circuits
from bluepysnap import Circuit
import numpy as np

for _inst in grid_scan.coordinate_instances():
    cfg = grid_scan.output_root / str(_inst.neuron_set.neuron_ids) / "circuit_config.json"
    c = Circuit(cfg)
    nodes = c.nodes['hippocampus_neurons']
    mtypes = np.unique(nodes.get(properties=['mtype']), return_counts=True)
    mt_hist = [f"{_c}x{_mt}" for _mt, _c in zip(*mtypes)]
    print(f"{_inst.neuron_set.neuron_ids}: {nodes.size} neurons, {c.edges["hippocampus_neurons__hippocampus_neurons__chemical_synapse"].size} synapses, m-types: {mt_hist}")
